In [31]:
import numpy as np
import csv
import pandas as pd

In [32]:
# dedicated runtime
# appRuntime = np.load('../dedicated_runtime/app_dedicated_rt.npy').item()

In [33]:
#
# read runtime in ../02_dedicate
#
app_at_gpu0 = np.load('../../02_dedicate/homedesktop-gpu0.npy').item()
print app_at_gpu0

{'rodinia_b+tree': 1.5293519496917725, 'cudasdk_reduction': 1.1114969253540039, 'cudasdk_convolutionFFT2D': 3.5076291561126709, 'shoc_lev1BFS': 0.70082902908325195, 'rodinia_lavaMD': 0.96436595916748047, 'poly_gemm': 1.3004980087280273, 'rodinia_hybridsort': 0.87156295776367188, 'cudasdk_MCEstimatePiInlineQ': 0.77948307991027832, 'cudasdk_MCEstimatePiInlineP': 0.6841280460357666, 'cudasdk_shflscan': 0.64646100997924805, 'rodinia_dwt2d': 0.70926189422607422, 'cudasdk_vectorAdd': 0.61124396324157715, 'cudasdk_MCEstimatePiP': 0.81639719009399414, 'cudasdk_radixSortThrust': 1.4740438461303711, 'shoc_lev1md5hash': 0.68527078628540039, 'parboil_bfs': 0.88222908973693848, 'poly_covariance': 51.159648895263672, 'shoc_lev1fft': 0.71208500862121582, 'cudasdk_segmentationTreeThrust': 2.9541587829589844, 'cudasdk_simpleCUBLAS': 0.79222989082336426, 'parboil_sgemm': 0.64826798439025879, 'cudasdk_concurrentKernels': 0.64674997329711914, 'cudasdk_fastWalshTransform': 3.1675691604614258, 'cudasdk_Sobo

In [34]:
app_at_gpu1 = np.load('../../02_dedicate/homedesktop-gpu1.npy').item()
print app_at_gpu1

{'rodinia_b+tree': 1.0812499523162842, 'cudasdk_reduction': 0.68117094039916992, 'cudasdk_convolutionFFT2D': 3.0999901294708252, 'shoc_lev1BFS': 0.24061894416809082, 'rodinia_lavaMD': 0.43408584594726562, 'poly_gemm': 0.86061501502990723, 'rodinia_hybridsort': 0.48058485984802246, 'cudasdk_MCEstimatePiInlineQ': 0.24531793594360352, 'cudasdk_MCEstimatePiInlineP': 0.26801586151123047, 'cudasdk_shflscan': 0.25724601745605469, 'rodinia_dwt2d': 0.25281000137329102, 'cudasdk_vectorAdd': 0.23250794410705566, 'cudasdk_MCEstimatePiP': 0.28968906402587891, 'cudasdk_radixSortThrust': 1.0569000244140625, 'shoc_lev1md5hash': 0.31501007080078125, 'parboil_bfs': 0.38708305358886719, 'poly_covariance': 53.46834397315979, 'shoc_lev1fft': 0.24262189865112305, 'cudasdk_segmentationTreeThrust': 2.6605408191680908, 'cudasdk_simpleCUBLAS': 0.38605713844299316, 'parboil_sgemm': 0.24231886863708496, 'cudasdk_concurrentKernels': 0.24102401733398438, 'cudasdk_fastWalshTransform': 2.7304399013519287, 'cudasdk_So

In [35]:
print type(app_at_gpu1)

<type 'dict'>


In [36]:
appRuntime = {}
for app, gpu0_rt in app_at_gpu0.iteritems():
    gpu1_rt = app_at_gpu1[app]
    appRuntime[app] = min(gpu0_rt, gpu1_rt)

In [37]:
# for app, app_rt in appRuntime.iteritems():
#     print app, app_rt

In [38]:
def read_timing_df(curfile):
    """read timing traces in csv file, offset the trace with the starting pos, return the dataframe"""
    df_current = pd.read_csv(curfile)
    offset = df_current['start'].min()  # find the starting timing, and offset it
    df_current['start'] = df_current['start'].apply(lambda x: x - offset)
    df_current['end'] = df_current['end'].apply(lambda x: x - offset)
    return df_current

def compare_with_dedicatedruntime(df, appRuntime):
    """
    read the timing traces in dataframe (after read_timing_df()).
    compute the speedup over the dedicated runtime
    return speedup list
    """
    dedicated_total,  corun_total = 0.0, 0.0
    speedup_list = []

    # columns => ['jobID', 'appName', 'start', 'end', 'duration (s)']
    for index, row in df.iterrows():
        app_name, app_endT = row['appName'], row['end']
        app_rt_dedicate = float(appRuntime[app_name])  # dedicated runtime
        app_endT = float(app_endT)
        if corun_total < app_endT: corun_total = app_endT
        dedicated_total += app_rt_dedicate
        speedup = dedicated_total / corun_total
        speedup_list.append(speedup)   
    return speedup_list

In [39]:
len(appRuntime)

75

In [40]:
appRuntime

{'cudasdk_BlackScholes': 1.9693429470062256,
 'cudasdk_FDTD3d': 9.9745099544525146,
 'cudasdk_MCEstimatePiInlineP': 0.26801586151123047,
 'cudasdk_MCEstimatePiInlineQ': 0.24531793594360352,
 'cudasdk_MCEstimatePiP': 0.28968906402587891,
 'cudasdk_MCEstimatePiQ': 0.27803206443786621,
 'cudasdk_MCSingleAsianOptionP': 0.2678530216217041,
 'cudasdk_SobolQRNG': 0.31908392906188965,
 'cudasdk_batchCUBLAS': 0.37936806678771973,
 'cudasdk_binomialOptions': 22.217447996139526,
 'cudasdk_boxFilterNPP': 0.25980210304260254,
 'cudasdk_c++11Cuda': 0.84449291229248047,
 'cudasdk_concurrentKernels': 0.24102401733398438,
 'cudasdk_convolutionFFT2D': 3.0999901294708252,
 'cudasdk_convolutionSeparable': 1.5115499496459961,
 'cudasdk_convolutionTexture': 0.90807485580444336,
 'cudasdk_dct8x8': 0.4726259708404541,
 'cudasdk_dwtHaar1D': 0.23399615287780762,
 'cudasdk_dxtc': 0.26365184783935547,
 'cudasdk_eigenvalues': 2.1058349609375,
 'cudasdk_fastWalshTransform': 2.7304399013519287,
 'cudasdk_interval': 

In [41]:
def compute_antt(inputcsv):
    """Read app status table in csv format, compute the ANTT"""
    
    df = read_timing_df(inputcsv)
    
    #
    # 1) sort the dataframe by 'end'
    #
    sorted_df = df.sort_values(by=['end'])
    
    
    #
    # 2) compute the ntt for each app
    # 
    antt_list = []
    for idx, row in sorted_df.iterrows():
        appName = row['appName']
        T_corun = float(row['duration (s)'])
        T_dedicate = float(appRuntime[appName])

        tmp = T_corun / T_dedicate
        #print appName, T_corun, T_dedicate, tmp
        antt_list.append(tmp)

    #
    # 3) in antt list, add current value with prev
    # 
    antt = [0 for i in xrange(len(antt_list))]
    for i, v in enumerate(antt_list):
        if i == 0:
            antt[0] = v
        else:
            antt[i] = antt[i-1] + v

    # 4) final step:  1 / N on each data sample
    antt_seq = []
    for i, v in enumerate(antt):
        #print i, v
        N = i + 1
        current_antt = v / N
        #print current_antt
        antt_seq.append(current_antt)
    
    return antt_seq
    

### read traces

In [42]:
# we have featall/feat64/feat42/feat42/feat26/feat18/feat14/feat12/featMystic, 
# to compare with fcfs

In [49]:
prefix = "ll"
for sid in [1,2,3]:
    for cid in [2,4,6,8]:
        inputcsv = prefix + "-s" + str(sid) + "-c" + str(cid) + ".csv"
        antt = compute_antt(inputcsv)
        print inputcsv, antt[-1]
    print "\n"

ll-s1-c2.csv 2.05952154258
ll-s1-c4.csv 5.98926415741
ll-s1-c6.csv 11.5025856958
ll-s1-c8.csv 17.2184122228


ll-s2-c2.csv 3.76602021664
ll-s2-c4.csv 6.64194815203
ll-s2-c6.csv 10.8012008455
ll-s2-c8.csv 16.223022544


ll-s3-c2.csv 2.50587493215
ll-s3-c4.csv 6.72024758485
ll-s3-c6.csv 11.016247405
ll-s3-c8.csv 19.4954863




In [50]:
prefix = "rr"
for sid in [1,2,3]:
    for cid in [2,4,6,8]:
        inputcsv = prefix + "-s" + str(sid) + "-c" + str(cid) + ".csv"
        antt = compute_antt(inputcsv)
        print inputcsv, antt[-1]
    print "\n"

rr-s1-c2.csv 2.70313684557
rr-s1-c4.csv 5.67045500764
rr-s1-c6.csv 9.71917695117
rr-s1-c8.csv 13.279964691


rr-s2-c2.csv 2.42798229713
rr-s2-c4.csv 5.47553718805
rr-s2-c6.csv 11.0859911468
rr-s2-c8.csv 15.3662365971


rr-s3-c2.csv 3.00660359346
rr-s3-c4.csv 4.05191473127
rr-s3-c6.csv 6.68405406229
rr-s3-c8.csv 10.75759219




In [51]:
prefix = "xrbfeatAll"
for sid in [1,2,3]:
    for cid in [2,4,6,8]:
        inputcsv = prefix + "-s" + str(sid) + "-c" + str(cid) + ".csv"
        antt = compute_antt(inputcsv)
        print inputcsv, antt[-1]
    print "\n"

xrbfeatAll-s1-c2.csv 1.63654723186
xrbfeatAll-s1-c4.csv 3.47270745338
xrbfeatAll-s1-c6.csv 6.75836415539
xrbfeatAll-s1-c8.csv 9.7571460936


xrbfeatAll-s2-c2.csv 1.79221184308
xrbfeatAll-s2-c4.csv 3.1794341222
xrbfeatAll-s2-c6.csv 6.62755224504
xrbfeatAll-s2-c8.csv 10.6047763391


xrbfeatAll-s3-c2.csv 1.44648486756
xrbfeatAll-s3-c4.csv 2.89107505696
xrbfeatAll-s3-c6.csv 5.85886637627
xrbfeatAll-s3-c8.csv 8.69414454215




In [52]:
prefix = "xrbfeat9"
for sid in [1,2,3]:
    for cid in [2,4,6,8]:
        inputcsv = prefix + "-s" + str(sid) + "-c" + str(cid) + ".csv"
        antt = compute_antt(inputcsv)
        print inputcsv, antt[-1]
    print "\n"

xrbfeat9-s1-c2.csv 1.64799588036
xrbfeat9-s1-c4.csv 2.64912525823
xrbfeat9-s1-c6.csv 5.80264664274
xrbfeat9-s1-c8.csv 8.39025025472


xrbfeat9-s2-c2.csv 1.96629013899
xrbfeat9-s2-c4.csv 2.45641208786
xrbfeat9-s2-c6.csv 5.04867421294
xrbfeat9-s2-c8.csv 8.77101923037


xrbfeat9-s3-c2.csv 1.36731079596
xrbfeat9-s3-c4.csv 2.75049903036
xrbfeat9-s3-c6.csv 4.92372674543
xrbfeat9-s3-c8.csv 9.49869967856


